
# Zcash Observatory data

Mitchell Krawiec-Thayer and Pranav Thirunavukkarasu

June 2020

Observatory R & D at Insight supported by the Zcash Foundation


## Parameters

In [18]:
path_to_files = '.'

## Import libraries

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os;

## Import data

In [20]:
# What are the nearby directories
folders = [x[0] for x in os.walk(path_to_files)]

In [106]:
# global_df = pd.DataFrame()
first_time = 1
for f in range(len(folders)):
    this_folder_raw = folders[f]
    if not (this_folder_raw[0:3] == './.' or this_folder_raw == '.'):
        node_name = this_folder_raw[2::]
        file_name = os.path.join(this_folder_raw,'inv_v1.csv')
        this_df = pd.read_csv(file_name)
        
        # Some feature engineering
        min_time = this_df.groupby('Hash').min()
        min_time = min_time.rename(columns={"Validated_Time": "min_time", "Peer_IP": "min_IP", "Hash":'block_hash'})
        max_time = this_df.groupby('Hash').max()
        max_time = max_time.rename(columns={"Validated_Time": "max_time", "Peer_IP": "max_IP", "Hash":'block_hash'})
        combined = pd.concat([min_time, max_time], axis=1)
        combined['prop_time'] = combined['max_time']-combined['min_time']
        
        if first_time:
            global_df = combined.filter(['block_hash','prop_time'], axis=1)
            global_df = ## Ahhh need to fold in more features
        
        first_time = 0
        

In [107]:
global_df

,prop_time
Hash,
000000000000b86df2619ffbde1d3de76bdf1cc9c73b4acc91dccad6f234ec71,27.129
000000000000bc65600076a474767b0201c02fac53adb217dcafb8f44a3de6e4,12.173
000000000000c32bff2dcb5497f93438548bf10ef1e7ac541b0b0e81a100e1f1,61.857
0000000000016608d92021f3056e25742fc063ff7409fbd80f9cf0d290516a16,5.704
000000000002dd1be009ee3ee605f3c109ed3cc5514772f1169a7766e94bf77b,269.665
00000000000362bf8fac3df98968ff0169900a89a8d46bb765099ef7fa5662dd,8.322
00000000000382b9c7d130c4af4049af302fa90ff25548008b0056da4676df10,62.638
0000000000062e4896b67aa139d8fbdd6ccc87353f0de48353066453ec39cb09,3.057
000000000006e68de16fac5507de02575bdea37146c4e96d84d42965d54d9843,8.289


In [96]:
min_time = this_df.groupby('Hash').min()
min_time = min_time.rename(columns={"Validated_Time": "min_time", "Peer_IP": "min_IP"})
max_time = this_df.groupby('Hash').max()
max_time = max_time.rename(columns={"Validated_Time": "max_time", "Peer_IP": "max_IP"})
combined = pd.concat([min_time, max_time], axis=1)
combined['prop_time'] = combined['max_time']-combined['min_time']